In [1]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import gc 
#from spacy.tokenizer import Tokenizer
#from spacy.lang.en import English
import re

In [2]:
#hide
#!pip install -Uqq fastbook
#hide
from fastbook import *
from IPython.display import display,HTML
from fastai.text.all import *
from fastai.text.all import Tokenizer as Tokenize

import fastbook
fastbook.setup_book()

C:\Users\tonyl\anaconda3\lib\site-packages\fastbook\__init__.py:18: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [6]:
pd.options.display.max_colwidth = 150
df = pd.read_csv("cleand_tweet.csv")

# df.tweet = df.tweet.str.split(pat=': ').str[-1]
# df.tweet.to_csv("df_tweet.csv")

In [167]:
df = df[~df['tweet'].str.contains('http')].reset_index()
len(df)

19500

In [168]:
df['tweet'][22:24]

22    " fuck no that bitch dont even suck dick " &#128514;&#128514;&#128514; the Kermit videos bout to fuck IG up
23                                                                 " her pussy lips like Heaven doors " &#128524;
Name: tweet, dtype: object

In [175]:
test = df.tweet[51]
rnd_name = str(names.get_first_name())
matched = re.sub('@.+?(\b|\s|$)', rnd_name+str(' '), test)
matched
test_2 = 'How u gone bring ur side bitch to a game where &#128553;You know Ya gf fri&#128553;ends at ?! &#128553;&#128553;" I SWEAR!!!!!'
matched_2 = re.sub('&#.+?;', rnd_name+str(''), test_2)
matched_2

'How u gone bring ur side bitch to a game where DorothyYou know Ya gf friDorothyends at ?! DorothyDorothy" I SWEAR!!!!!'

In [180]:
import names

for i in range(len(df['tweet'])):
    rnd_name = str(names.get_first_name())
    df['tweet'][i]  = re.sub('@.+?(\b|\s|$)', rnd_name+str(' '), df['tweet'][i])
    df['tweet'][i]  = re.sub('&.+?;', '', df['tweet'][i])

<ipython-input-180-480d5481ee68>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i]  = re.sub('@.+?(\b|\s|$)', rnd_name+str(' '), df['tweet'][i])
<ipython-input-180-480d5481ee68>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i]  = re.sub('&.+?;', '', df['tweet'][i])


In [184]:
df.to_csv("cleand_tweet.csv")

In [7]:
tokenizer = WordTokenizer()
tkn = Tokenize(tokenizer)

In [8]:
print(df.tweet[2])
print(coll_repr(tkn(df.tweet[2]), 30))

Alma she look like a tranny
(#8) ['xxbos','xxmaj','alma','she','look','like','a','tranny']


In [9]:
toks200 = df.tweet[:200].map(tkn)
print(coll_repr(toks200[0], 30))

(#18) ['xxbos','boy','dats','cold','…','tyga','dwn','bad','for','cuffin','dat','hoe','in','the','1st','place','!','!']


In [10]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,50)

'(#200) [\'xxunk\',\'xxpad\',\'xxbos\',\'xxeos\',\'xxfld\',\'xxrep\',\'xxwrep\',\'xxup\',\'xxmaj\',\'"\',\'i\',\'bitch\',\'.\',\'a\',\'you\',\'the\',\'that\',\'to\',\'!\',"n\'t",\'bitches\',\'hoes\',\'do\',\'my\',\'like\',\'me\',\',\',\'pussy\',\'be\',\'for\',\'?\',\'on\',\'up\',\'is\',\'in\',\'and\',\'it\',\'with\',\'get\',\'these\',\'if\',\'…\',"\'m",\'of\',\'shit\',\'..\',\'nt\',\'all\',\'fuck\',\'but\'...]'

In [11]:
nums200 = toks200.map(num)

In [12]:
dl = LMDataLoader(nums200)

In [13]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 57]), torch.Size([64, 57]))

In [14]:
## Checking
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxunk xxunk xxunk … xxunk xxunk bad for xxunk xxunk hoe in the xxunk xxunk ! ! xxbos xxmaj'

In [15]:
dls = TextDataLoaders.from_df(df, text_col='tweet', is_lm=True)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


C:\Users\tonyl\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [16]:

learn = language_model_learner(
    dls, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [195]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Ti'

In [196]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.876323,4.533437,0.224273,93.077942,00:11


In [ ]:
learn.save('10epoch')

In [20]:
learner = torch.load('models/finetuned.pth')

In [27]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.350114,4.937599,0.209527,139.435028,00:13
1,4.819173,4.497620,0.236074,89.803108,00:12
2,4.448807,4.331578,0.257218,76.064201,00:12
3,4.159441,4.299310,0.261147,73.648941,00:12
4,3.830266,4.357162,0.257637,78.035355,00:12
5,3.500044,4.462615,0.253428,86.714012,00:12
6,3.131141,4.609733,0.250200,100.457336,00:12
7,2.820148,4.754954,0.245800,116.158295,00:12
8,2.587677,4.842947,0.244403,126.842667,00:12
9,2.463525,4.874281,0.243503,130.880066,00:12


In [28]:
learn.save_encoder('finetuned')

In [35]:
TEXT = "My friend Truman"
N_WORDS = 50
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [36]:
print("Inputted sentence: "+TEXT)
print("\nGenerated sentences...")
print("\n".join(preds))

Inputted sentence: My friend Truman

Generated sentences...
My friend Xxunk got all the bitches On some real shit , i think i been in a fucking crib .. June Calm down you little Asian bitch … . When people say " god only listen " against the word hoe , they argue
My friend Xxunk is a fucking bitch . Who 's your mom ? Why is Washington Redskins no pride ? Fuck . Mom ! ! Steve why ? Fuck u stupid hoe I 'm fuckin up and not everyone bout Lmfa
